## Natural Disaster

The purpose of this project is to use regression analysis to estimate natural disaster casulties and damage cost. This will be of great value in emergency planning and resource allocation in the event of a natural disaster. 

**Datasets Used**
* *EM-DAT: Natural Disasters Database.*
    Historical earthquake, storm and flood data for years 2000-2019

* *Global population estimates by longitude and latitude* For years 2000, 2005, 2010, 2015 and 2020(estimate)
    
* *USGS Earthquake Database.* Historical earthquake catalog with details such as earthquake magnitude, intensity and depth




**References**
* Center for International Earth Science Information Network - CIESIN - Columbia University. 2018. Gridded Population of the World, Version 4 (GPWv4): Administrative Unit Center Points with Population Estimates, Revision 11. Palisades, NY: NASA Socioeconomic Data and Applications Center (SEDAC). https://doi.org/10.7927/H4BC3WMT. Accessed June 9, 2019

    


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
#from pyspark import SparkContext, SparkConf
#sc =SparkContext()
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Importing Datasets

In [2]:
earthquake_data = pd.read_excel("earthquake_disaster_info_2000.xlsx")
earthquake_details = pd.read_excel("earthquake_details.xlsx")

In [3]:
print("Disaster Data: {}, Population Data: {}, Earthquake Date: {}".format(
    earthquake_data.shape, pop_data.shape, earthquake_details.shape))

NameError: name 'pop_data' is not defined

In [ ]:
earthquake_data.head(3)

In [ ]:
pop_data.head(3)

In [ ]:
earthquake_details.head(3)

## Exploratory Analysis

In [ ]:
earthquake_data.describe(include='all')

In [ ]:
earthquake_data['Country'].value_counts().head(10).plot(kind='bar', figsize=(15,3))

In [ ]:
arr_magnitude = earthquake_data['Magnitude value'].values
sns.set(color_codes=True)
plt.figure(figsize=(15,5))
sns.distplot(arr_magnitude, bins=20)
plt.title("Distribution of Magnitude Values (Ritcher Scale)")

#### Correlation between Magnitude Value and Total Death?

In [ ]:
arr_death = earthquake_data['Total deaths'].values
sns.jointplot(arr_magnitude, arr_death)

There is no direct linear correlation between magnitude value and total death. A magnitude value of 9.0+ does not correlate to higher death rate. Why is that? Earthquake damage depends moreso on what area is hit. If an unpopulated region is struck, there will be low loss of life or property. However, if the earthquake is struck in a highly densed area, there will be more casualities. Therefore, magnitude alone cannot be used to predict death rate. Another important factor is location and location population/density. I'll need to somehow incorporate the **Total Affected** data into my model.

## Joining Datasets

To understand the strength and severity of earthquakes, I need to not only look at the **magnitutde** of an earthquake but also its **intensity** and **depth**. The EM-DAT data sets stores information about the magnitude of the earthquake, but I need to join it with the USGS earthquake datasets to pull the depth and intensity. 

In [ ]:
## splitting earthquake details 'time' column into year and month
details_join = earthquake_details.copy()
details_join = details_join[['time', 'latitude', 'longitude', 'depth', 'mag','place']]
details_join['year'] = details_join['time'].apply(lambda x: x.split('-')[0]).astype('int')
details_join['month'] = details_join['time'].apply(lambda x: x.split('-')[1]).astype('int')
details_join['day'] = details_join['time'].apply(lambda x: x.split('-')[2][:2]).astype('int')
details_join['latitude_round'] = round(details_join['latitude'])
details_join['longitude_round'] = round(details_join['longitude'])

def split_country(place):
    split_list = place.split(', ')
    list_len = len(split_list)
    if list_len > 1: return split_list[1]
    else: return split_list[0]
    
details_join['country'] = details_join['place'].astype(str).apply(lambda x: split_country(x))
remove_cardinal = ["southeastern ","northeastern ","southwestern ","northwestern ",
                   "western ","eastern ","southern ","northern ","near the coast of central "]
for loc in remove_cardinal:
    details_join['country'].replace(loc,"",regex=True,inplace=True)
details_join.head(3)

In [ ]:
## split natural disaster 'Start date' column into year, month, day
earthquake_feature_data = earthquake_data[['Start date','ISO','Country','Latitude','Longitude',
                                  'Magnitude value','Total deaths']].copy()
earthquake_feature_data['Month'] = earthquake_data['Start date'].apply(lambda x: x.split('/')[1]).astype('int')
earthquake_feature_data['Year'] = earthquake_data['Start date'].apply(lambda x: x.split('/')[2]).astype('int')
earthquake_feature_data['Day'] = earthquake_data['Start date'].apply(lambda x: x.split('/')[0]).astype('int')
earthquake_feature_data['Latitude_round'] = round(earthquake_data['Latitude'])
earthquake_feature_data['Longitude_round'] = round(earthquake_data['Longitude'])

## standardize country names
rename_country = [" \(the\)"," \(Islamic Republic of\)"," \(Province of China\)",
                  ", United Republic of"," \(the Democratic Republic of the\)", " Federation"]
earthquake_feature_data['Country'].replace("Russian Federation","Russia",regex=True,inplace=True)
for country in rename_country:
    earthquake_feature_data['Country'].replace(country,"",regex=True,inplace=True)

print(earthquake_feature_data.info())
print(earthquake_feature_data.describe(include='all'))

In [ ]:
earthquake_feature_data.head(3)

In [ ]:
details_join.head(3)

In [ ]:
earthquake_merge_data = earthquake_feature_data.copy()
earthquake_merge_data.reset_index(inplace=True)

In [ ]:
## merge attempt 1: Exact Date and Location
def merge_data(earthquake,details,earth_col,details_col):
    details_dist = details.drop_duplicates(subset=details_col)
    df_merge = pd.merge(earthquake,details_dist,how='left',
                       left_on=earth_col,right_on=details_col)
    null = df_merge['mag'].isnull()
    print("Number of un-merged rows left: {}".format(null.sum()))
    null_index = df_merge[null]['index'].values
    print("Un-merged index rows: {}".format(null_index))
    full_df=df_merge[~null]
    print("Merge dataframe shape: {}".format(full_df.shape))
    return full_df,null_index


merged_df1, null_index = merge_data(earthquake_merge_data,details_join,
                                   ['Latitude_round','Longitude_round','Year','Month','Day'],
                                   ['latitude_round','longitude_round','year','month','day'])
earthquake_merge_df = earthquake_merge_data.iloc[null_index,:]
print(earthquake_merge_df.shape)

In [ ]:
## merge attempt 2: Exact Date and Country
merged_df2, null_index2 = merge_data(earthquake_merge_df,details_join,
                                   ['Country','Year','Month','Day'],
                                   ['country','year','month','day'])
earthquake_merge_df2 = earthquake_merge_data.iloc[null_index2,:]
print(earthquake_merge_df2.shape)

In [ ]:
## merge attempt 3: Date -1 and Country
earthquake_merge_df2_date_adj = earthquake_merge_df2.copy()
earthquake_merge_df2_date_adj['Day'] = earthquake_merge_df2_date_adj['Day'].astype(int) - 1
merged_df3, null_index3 = merge_data(earthquake_merge_df2_date_adj,details_join,
                                   ['Country','Year','Month','Day'],
                                   ['country','year','month','day'])
earthquake_merge_df3 = earthquake_merge_data.iloc[null_index3,:]

## merge attempt 4: Date +1 and Country
earthquake_merge_df3_date_adj = earthquake_merge_df3.copy()
earthquake_merge_df3_date_adj['Day'] = earthquake_merge_df3_date_adj['Day'].astype(int) + 1
merged_df4, null_index4 = merge_data(earthquake_merge_df3_date_adj,details_join,
                                   ['Country','Year','Month','Day'],
                                   ['country','year','month','day'])
earthquake_merge_df4 = earthquake_merge_data.iloc[null_index4,:]

In [ ]:
merged_df5, null_index5 = merge_data(earthquake_merge_df4,details_join,
                                   ['Latitude_round','Longitude_round','Year','Month'],
                                   ['latitude_round','longitude_round','year','month'])
earthquake_merge_df5 = earthquake_merge_data.iloc[null_index5,:]
earthquake_merge_df5

In [ ]:
drop_rows = [28,476,343,352,318,307,161,96,136,151,165,185,190]
earthquake_merge_df6 = earthquake_merge_df5[~earthquake_merge_df5.index.isin(drop_rows)]
earthquake_merge_df6

In [ ]:
manual_merge = [(467,21165),(398,107067)]
def join_row(index1,index2):
    row1 = earthquake_merge_df6[earthquake_merge_df6.index==index1]
    print(row1.shape)
    row2 = details_join[details_join.index==index2]
    print(row2.shape)
    row = row1.join(row2)
    print(row)
    return row
manual_merge_list = [join_row(pair[0],pair[1]) for pair in manual_merge]
pd.concat(manual_merge_list,axis=0)

In [ ]:
merged_df = pd.concat([merged_df1,merged_df2,merged_df3,merged_df4,merged_df5])
print(merged_df.shape)
print(merged_df.columns)
merged_df.head(3)

## Finding Outliers

In [ ]:
merged_df['diff_mag'] = merged_df['Magnitude value'] - merged_df['mag']
merged_df['diff_mag'].plot(kind='hist',figsize=(10,3))

In [ ]:
merged_df[merged_df['diff_mag']>=2]